<a href="https://colab.research.google.com/github/AP-047/Rational-Function-Classifier/blob/main/notebooks/02_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Training